<a href="https://colab.research.google.com/github/dannynacker/strobe_entrainment_periodicity_MSc/blob/main/roXiva_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import random

def poisson_flash_times(frequency, duration, min_interval=0.025):
    """Generates Poisson-distributed flash onset times and durations similar to Lionel's logic."""
    cycle_length = 1.0 / frequency  # Time per cycle
    on_duration = cycle_length / 2  # Fixed on-duration (half-cycle duration)
    expected_flashes = int(duration * frequency)  # Ensuring Fe matches target frequency

    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length)
        time += interval
        if time + on_duration < duration:
            times.append((time, on_duration))  # Store onset time and duration

    return times

def regularise_strobe(signal, rmode=3, min_interval=0.025):
    """Regularizes strobe sequence by handling overlaps."""
    if rmode == 3:
        output_signal = []
        last_flash_end = -np.inf
        for onset, duration in signal:
            if onset >= last_flash_end + min_interval:
                output_signal.append((onset, duration))
                last_flash_end = onset + duration
        return output_signal
    return signal

def generate_strobe_sequence():
    """User input function to generate a strobe sequence file, ensuring Poisson-distributed STP entries with correct formatting."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_f = float(input("Enter start frequency (Hz): "))
    end_f = float(input("Enter end frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    led_config = [input(f"LED Set {i+1} (1=On, 0=Off): ") for i in range(4)]

    flash_times = poisson_flash_times((start_f + end_f) / 2, duration)
    flash_times = regularise_strobe(flash_times)

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    previous_onset = 0
    min_interval = 0.025  # Ensure min_interval is defined
    num_flashes = len(flash_times)

    for i, (onset, on_duration) in enumerate(flash_times):
        step_duration = max(onset - previous_onset, min_interval)  # Ensure valid timing
        previous_onset = onset + on_duration  # Update previous onset to avoid fixed intervals

        # Interpolate luminance and duty cycle
        current_l = start_l + (end_l - start_l) * (i / num_flashes)
        current_d = start_d + (end_d - start_d) * (i / num_flashes)

        step_string = f'{step_duration:.3f},1,{start_f:.2f},{end_f:.2f},{int(current_d)},{int(current_d)},{" ,".join(led_config)},{int(current_l)},{int(current_l)}'
        output.append(f'STP"{step_string}"')

    with open("strobe_sequence.txt", "w") as f:
        f.write('\n'.join(output))

    print("Generated sequence saved as strobe_sequence.txt")

# Run the generator
generate_strobe_sequence()

Choose stimulation type (Periodic/Aperiodic): Aperiodic
Enter total sequence duration (seconds): 5
Enter start frequency (Hz): 10
Enter end frequency (Hz): 15
Enter start luminance (0-100): 25
Enter end luminance (0-100): 75
Enter start duty cycle (1-99): 25
Enter end duty cycle (1-99): 75
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
Generated sequence saved as strobe_sequence.txt


In [1]:
import numpy as np
import random

def poisson_flash_times(frequency, duration, min_interval=0.025):
    """Generates Poisson-distributed flash onset times and durations matching MATLAB's logic."""
    cycle_length = 1.0 / frequency  # Time per cycle
    on_duration = cycle_length / 2  # Fixed on-duration (half-cycle duration)
    expected_flashes = int(duration * frequency)  # Ensuring Fe matches target frequency

    times = []
    time = 0
    while len(times) < expected_flashes and time < duration:
        interval = np.random.exponential(cycle_length)
        time += interval
        if time + on_duration < duration:
            times.append((time, on_duration))  # Store onset time and duration

    return times

def regularise_strobe(signal, rmode=3, min_interval=0.025):
    """Regularizes strobe sequence by handling overlaps, mimicking MATLAB's regularisation logic."""
    if rmode == 3:
        output_signal = []
        last_flash_end = -np.inf
        for onset, duration in signal:
            if onset >= last_flash_end + min_interval:
                output_signal.append((onset, duration))
                last_flash_end = onset + duration
        return output_signal
    return signal

def generate_strobe_sequence():
    """Interactive function to generate a strobe sequence file, ensuring Poisson-distributed STP entries with correct formatting."""
    stimulation = input("Choose stimulation type (Periodic/Aperiodic): ").strip()
    duration = float(input("Enter total sequence duration (seconds): "))
    start_f = float(input("Enter start frequency (Hz): "))
    end_f = float(input("Enter end frequency (Hz): "))
    start_l = int(input("Enter start luminance (0-100): "))
    end_l = int(input("Enter end luminance (0-100): "))
    start_d = int(input("Enter start duty cycle (1-99): "))
    end_d = int(input("Enter end duty cycle (1-99): "))
    wave_type = input("Enter wave type (Square/Sine): ").strip()
    led_config = [input(f"LED Set {i+1} (1=On, 0=Off): ") for i in range(4)]

    flash_times = poisson_flash_times((start_f + end_f) / 2, duration)
    flash_times = regularise_strobe(flash_times)

    output = [f'TIM"00:00:{int(duration // 60):02}:{duration % 60:04.1f}"', f'DUR"{duration:.1f}"']

    previous_onset = 0
    min_interval = 0.025  # Ensure min_interval is defined
    num_flashes = len(flash_times)

    for i, (onset, on_duration) in enumerate(flash_times):
        step_duration = max(onset - previous_onset, min_interval)  # Ensure valid timing
        previous_onset = onset + on_duration  # Update previous onset to avoid fixed intervals

        # Interpolate luminance, duty cycle, and frequency
        current_l = start_l + (end_l - start_l) * (i / (num_flashes - 1))
        current_d = start_d + (end_d - start_d) * (i / (num_flashes - 1))
        current_f_start = start_f + (end_f - start_f) * (i / (num_flashes - 1))
        current_f_end = start_f + (end_f - start_f) * ((i + 1) / (num_flashes - 1))

        step_string = f'{step_duration:.3f},1,{current_f_start:.2f},{current_f_end:.2f},{int(current_d)},{int(current_d)},{" ,".join(led_config)},{int(current_l)},{int(current_l)}'
        output.append(f'STP"{step_string}"')

    with open("strobe_sequence.txt", "w") as f:
        f.write('\n'.join(output))

    print("Generated sequence saved as strobe_sequence.txt")

# Run the generator
generate_strobe_sequence()
1

Choose stimulation type (Periodic/Aperiodic): Aperiodic
Enter total sequence duration (seconds): 5
Enter start frequency (Hz): 10
Enter end frequency (Hz): 15
Enter start luminance (0-100): 25
Enter end luminance (0-100): 75
Enter start duty cycle (1-99): 25
Enter end duty cycle (1-99): 75
Enter wave type (Square/Sine): Square
LED Set 1 (1=On, 0=Off): 1
LED Set 2 (1=On, 0=Off): 1
LED Set 3 (1=On, 0=Off): 1
LED Set 4 (1=On, 0=Off): 1
Generated sequence saved as strobe_sequence.txt
